In [35]:
import pandas as pd
import numpy as np

In [36]:
# Preparation for reading data from csv file on Google Colab
# remove when reading the csv file locally from your computer
import os
from google.colab import drive
drive.mount('/content/drive')
os.chdir('/content/drive/MyDrive/HS/Intern/DABI2_Projekt_25S/DABI2_25S_public')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [37]:
orders = pd.read_parquet("orders.parquet")
op = pd.read_csv("order_products_denormalized.csv.zip")
tips = pd.read_csv("tips_public.csv")

In [38]:
orders.head()

,order_id,user_id,order_date
0,1374495,3,2024-03-31 14:05:18
1,444309,3,2024-04-09 19:45:51
2,3002854,3,2024-04-30 16:25:20
3,2037211,3,2024-05-20 18:31:28
4,2710558,3,2024-06-01 17:52:27


In [39]:
tips.head()

,Unnamed: 0,order_id,tip
0,26,1374495,True
1,27,444309,True
2,28,3002854,True
3,29,2037211,True
4,30,2710558,True


#### Aufgabe: Den Zusammenhang zwischen der Trinkgeldgabe in einer Bestellung und der Trinkgeldgabe in der vorhergehenden bestellung desselben Users ermitteln:

In [40]:
h=pd.merge(orders[['order_id','user_id','order_date']], tips[['order_id','tip']])
h.head(20)

,order_id,user_id,order_date,tip
0,1374495,3,2024-03-31 14:05:18,True
1,444309,3,2024-04-09 19:45:51,True
2,3002854,3,2024-04-30 16:25:20,True
3,2037211,3,2024-05-20 18:31:28,True
4,2710558,3,2024-06-01 17:52:27,True
5,1972919,3,2024-06-08 16:55:12,True
6,1839752,3,2024-06-15 15:04:06,False
7,3225766,3,2024-06-22 17:49:38,True
8,3160850,3,2024-06-29 16:49:34,False
9,676467,3,2024-07-16 16:27:44,True


#### Im Prinip lässt sich das mit der shift(n)-Funktion lösen, die eine Spalte eines Dataframes nach oben oder unten verschiebt. Der Dataframe muss dazu passend sortiert sein:

In [41]:
h = h.sort_values(by=['user_id','order_date']) # Passend sortieren
h['tip-1'] = h.tip.shift(1) # Um 1 Zeile nach unten verschieben
h.head(20)

,order_id,user_id,order_date,tip,tip-1
0,1374495,3,2024-03-31 14:05:18,True,NaN
1,444309,3,2024-04-09 19:45:51,True,True
2,3002854,3,2024-04-30 16:25:20,True,True
3,2037211,3,2024-05-20 18:31:28,True,True
4,2710558,3,2024-06-01 17:52:27,True,True
5,1972919,3,2024-06-08 16:55:12,True,True
6,1839752,3,2024-06-15 15:04:06,False,True
7,3225766,3,2024-06-22 17:49:38,True,False
8,3160850,3,2024-06-29 16:49:34,False,True
9,676467,3,2024-07-16 16:27:44,True,False


#### Das Ergebnis stimmt fast, aber z.B. in zeile 16 wurde der Wert der letzten Bestellung des vorigen users fälschlich zur ersten Bestellung des nächsten Users verschoben. Dies lässt sich verhindern, wenn man shift(1) nicht direkt auf der Spalte 'tip' ausführt, sondern ein groupby(['user_id']) zwischenschaltet:

In [50]:
h['tip_1'] = h[['user_id','tip']].groupby(['user_id']).shift(1)
h.head(20)

,order_id,user_id,order_date,tip,tip_1
0,1374495,3,2024-03-31 14:05:18,True,NaN
1,444309,3,2024-04-09 19:45:51,True,True
2,3002854,3,2024-04-30 16:25:20,True,True
3,2037211,3,2024-05-20 18:31:28,True,True
4,2710558,3,2024-06-01 17:52:27,True,True
5,1972919,3,2024-06-08 16:55:12,True,True
6,1839752,3,2024-06-15 15:04:06,False,True
7,3225766,3,2024-06-22 17:49:38,True,False
8,3160850,3,2024-06-29 16:49:34,False,True
9,676467,3,2024-07-16 16:27:44,True,False


#### Jetzt hat man zu jeder Bestellung in verschiedenen Spalten die Trinkgeldinformation zu *dieser* und zur *vorhergehenden* Bestellung desselben Users.

In [54]:
pd.crosstab(h.tip_1, h.tip)

tip,False,True
tip_1,,
False,451108,179322
True,178804,530334


In [45]:
orders['order_weekday'] = orders['order_date'].dt.weekday  # Montag=0, Sonntag=6
orders.head()

,order_id,user_id,order_date,order_weekday
0,1374495,3,2024-03-31 14:05:18,6
1,444309,3,2024-04-09 19:45:51,1
2,3002854,3,2024-04-30 16:25:20,1
3,2037211,3,2024-05-20 18:31:28,0
4,2710558,3,2024-06-01 17:52:27,5


In [46]:
h = pd.merge(orders[['order_id','user_id','order_date']], tips[['order_id','tip']])
h = h.sort_values(by=['user_id','order_date'])
# h['tip-1'] = h[['tip','user_id']].groupby('user_id').shift(1)
h.head()

,order_id,user_id,order_date,tip
0,1374495,3,2024-03-31 14:05:18,True
1,444309,3,2024-04-09 19:45:51,True
2,3002854,3,2024-04-30 16:25:20,True
3,2037211,3,2024-05-20 18:31:28,True
4,2710558,3,2024-06-01 17:52:27,True
